In [1]:
import os
from data_management import DataSource, DataManager
from file_management import (SiteHierarchies, IndexFile,  FormulaFile)
from formula_tables import (FormulaTable, FormulaTableType,
                            FormulaTableTypeSimple, FormulaTableTypeCalculus,
                            FormulaTableTypeFinancial)
from content import Formulas, Definitions
import pandas as pd
from collections import namedtuple

data_source = DataSource()
site_hierarchy = SiteHierarchies(data_source.site_hierarchies)
formulas_by_year = Formulas(data_source.formulas_by_year)
formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)
formulas_cumulative.is_cumulative = True

In [6]:
df = formulas_cumulative.to_dataframe()

df[(df['Subject'] == 'Year 7') & (df['Category'] == 'Surds')]

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment


In [24]:
by_year = formulas_by_year.to_dataframe()
by_year =  by_year.rename(columns={'Subject':'Dependency'})

return_value = by_year.merge(
    right = data_source.subject_dependencies,
    left_on=['State', 'Dependency'],
    right_on=['State', 'Dependency'])

return_value = return_value.drop('Dependency', axis='columns')
#Re-order cols
return_value = return_value[['State', 'Subject', 'Syllabus topic', 'Syllabus subtopic code',
       'Syllabus subtopic', 'Category', 'Subcategory_1', 'Subcategory_2',
       'Description', 'Group', 'Formula', 'On formula sheet', 'Proof required',
       'Comment', ]]


return_value = return_value[return_value['Subject'] == 'Year 12 Ext 1']

display(return_value[return_value['Category'] =='Surds'])

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment
68,NSW,Year 12 Ext 1,NaN,TBA,NaN,Surds,NaN,NaN,NaN,NaN,$ \sqrt{xy} = \sqrt{x} \times \sqrt{y} $,False,False,NaN
75,NSW,Year 12 Ext 1,NaN,TBA,NaN,Surds,NaN,NaN,NaN,NaN,$ \sqrt{\dfrac{x}{y}} = \dfrac{\sqrt{x}}{\sqrt...,False,False,NaN


In [50]:
subject_dependencies = data_source.subject_dependencies

subject_dependencies['Dependencies'] = subject_dependencies.apply(
    lambda x: (
        [x['Subject']] if pd.isnull(x['Dependencies']) 
        else [x['Subject']] + x['Dependencies'].split(',')
    ), 
    axis='columns')


display(subject_dependencies)

,State,Subject,Dependencies
0,NSW,Year 7,[Year 7]
1,NSW,Year 8,"[Year 8, Year 7]"
2,NSW,Year 9,"[Year 9, Year 7, Year 8]"
3,NSW,Year 10,"[Year 10, Year 7, Year 8, Year 9]"
4,NSW,Year 11 Adv,"[Year 11 Adv, Year 7, Year 8, Year 9, Year 10]"
5,NSW,Year 11 Ext 1,"[Year 11 Ext 1, Year 7, Year 8, Year 9, Yea..."
6,NSW,Year 12 Adv,"[Year 12 Adv, Year 7, Year 8, Year 9, Year ..."
7,NSW,Year 12 Ext 1,"[Year 12 Ext 1, Year 7, Year 8, Year 9, Yea..."
8,NSW,Year 12 Ext 2,"[Year 12 Ext 2, Year 7, Year 8, Year 9, Yea..."


In [2]:

def get_data():
    """Reads and returns various reequired data objects as a dictionary"""
    data_source = DataSource()
    site_hierarchies = SiteHierarchies(data_source.site_hierarchies)
    formulas_by_year = Formulas(data_source.formulas_by_year)
    formulas_cumulative = Formulas(data_source.formulas_by_year_cumulative)
    formulas_cumulative.is_cumulative = True
    return {'site_hierarchies': site_hierarchies,
            'formulas_by_year': formulas_by_year,
            'formulas_cumulative': formulas_cumulative}


x = get_data()
x['formulas_by_year'].to_dataframe()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Category,Subcategory_1,Subcategory_2,Description,Group,Formula,On formula sheet,Proof required,Comment
0,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Multiplying terms with same base,NaN,$a^m \times a^n = a^{m+n}$,False,False,NaN
1,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Dividing terms with same base,NaN,$a^m \div a^n = \dfrac{a^m}{a^n} = a^{m-n}$,False,False,NaN
2,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Power of a power,NaN,$(a^m)^n = a^{m \times n}$,False,False,NaN
3,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of products,NaN,$(ab)^n = a^nb^n$,False,False,NaN
4,NSW,Year 9,TBA,TBA,TBA,Indices,Index Laws,NaN,Powers of quotients,NaN,$\left(\dfrac{a}{b}\right)^n = \dfrac{a^n}{b^n}$,False,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Arithmetic sequence,Sum of first n terms,Sum of the first n terms of an arithmetic sequ...,NaN,$S_n = \dfrac{n}{2}[2a + (n-1)d] = \dfrac{n}{2...,True,True,NaN
68,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,Recursive definition,Recursive definition of a geometric sequence,NaN,$T_n = rT_{n-1}$,False,False,NaN
69,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,n-th term,n-th term of a geometric sequence,NaN,$T_n = ar^{n-1}$,True,True,NaN
70,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,Financial mathematics,Geometric sequence,Sum of first n terms,Sum of the first n terms of a geometric sequence,NaN,$S_n = \dfrac{a(1-r^n)}{1-r} = \dfrac{a(r^n-1)...,True,True,NaN


In [2]:
definitions = Definitions(data_source.definitions_by_year)

definitions.to_dataframe()

,State,Subject,Syllabus topic,Syllabus subtopic code,Syllabus subtopic,Definition
0,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,annuities
1,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,sequence
2,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,series
3,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,arithmetic sequence and series
4,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,common difference
5,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,geometric sequence and series
6,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,common ratio
7,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,present value
8,NSW,Year 12 Adv,Financial Mathematics,MA-M1,Modelling Financial Situations,future value


In [5]:
class AAA():
    
    def hello(self):
        print ('world')

class BBB(AAA):
    
    def hello(self):
        print ('you')    
    

x = BBB()
x.hello()

you
